<a href="https://colab.research.google.com/github/Ahmadtjk/Ahmadtjk/blob/main/AB%20test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A/B Testing Framework for Social Media Product Optimization

A/B testing (also known as split testing) is a randomized controlled experiment that involves showing two versions (A and B) of a single variable to different segments of the audience at the same time and measuring which version is more effective based on a predefined metric. For social media product optimization, this could involve testing different UI layouts, recommendation algorithms, notification strategies, or ad creatives.

### Core Components of an A/B Test:

1.  **Hypothesis Formulation**: Clearly define what you expect to happen and why. (e.g., "Changing the 'Like' button color from blue to green will increase click-through rate by 5%").
2.  **Metrics Definition**: Identify the Key Performance Indicators (KPIs) that will be used to measure the success of the experiment. These often include:
    *   **Engagement Metrics**: Click-Through Rate (CTR), likes, shares, comments, time spent on post/page.
    *   **Conversion Metrics**: Sign-ups, purchases, ad clicks, app installs.
    *   **Retention Metrics**: Repeat visits, churn rate.
3.  **Experiment Design**:
    *   **Control Group (A)**: The existing version or baseline.
    *   **Treatment Group (B)**: The new version with the proposed change.
    *   **Randomization**: Users are randomly assigned to either group to minimize bias.
    *   **Sample Size Calculation**: Determine the number of users needed in each group to detect a statistically significant difference (if one exists) with a certain level of confidence.
    *   **Duration**: How long will the experiment run?
4.  **Data Collection**: Implement tracking to record user interactions and events for both groups.
5.  **Statistical Analysis**: Compare the performance of Group A and Group B using statistical tests (e.g., z-tests, t-tests) to determine if the observed difference is statistically significant or due to random chance.
6.  **Interpretation and Decision**: Based on the statistical analysis, decide whether to implement the change, iterate on the design, or discard it.

In [2]:
import numpy as np
import scipy.stats as st
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportion_effectsize # Added this import
import pandas as pd

def calculate_sample_size(baseline_conversion_rate, minimum_detectable_effect, alpha=0.05, power=0.8):
    """
    Calculates the required sample size for an A/B test for proportions.

    Args:
        baseline_conversion_rate (float): The conversion rate of the control group (e.g., 0.10 for 10%).
        minimum_detectable_effect (float): The smallest difference in conversion rate you want to detect (e.g., 0.02 for a 2% absolute increase).
        alpha (float, optional): The significance level (Type I error rate). Defaults to 0.05.
        power (float, optional): The statistical power (1 - Type II error rate). Defaults to 0.8.

    Returns:
        int: The required sample size per group.
    """
    # Calculate the conversion rate for the treatment group
    treatment_conversion_rate = baseline_conversion_rate + minimum_detectable_effect

    # Calculate effect size using Cohen's h for proportions
    # effect_size = sms.proportion.proportion_effectsize(baseline_conversion_rate, treatment_conversion_rate)

    # Calculate sample size using statsmodels
    # For two proportions, we can use TTestIndPower.solve_power directly with prop_one and prop_two
    # Note: statsmodels solve_power for proportions expects two proportions directly
    n = sms.NormalIndPower().solve_power(
        effect_size=proportion_effectsize(baseline_conversion_rate, treatment_conversion_rate), # Modified this line
        alpha=alpha,
        power=power,
        alternative='two-sided'
    )

    return int(np.ceil(n))

# Example Usage:
# Let's say your current click-through rate (CTR) is 5% (0.05)
# You want to detect a minimum absolute increase of 0.5% (0.005)
# With a significance level (alpha) of 0.05 and power of 0.8

baseline_ctr = 0.05
min_detectable_increase = 0.005 # absolute increase

sample_size_per_group = calculate_sample_size(baseline_ctr, min_detectable_increase)
print(f"Required sample size per group: {sample_size_per_group}")

baseline_conversion_rate = 0.10
min_detectable_effect = 0.01 # An absolute increase of 1% (from 10% to 11%)

sample_size_per_group_conv = calculate_sample_size(baseline_conversion_rate, min_detectable_effect)
print(f"Required sample size per group for conversion: {sample_size_per_group_conv}")

Required sample size per group: 31218
Required sample size per group for conversion: 14745


# Task
Simulate A/B test data for control and treatment groups, using the calculated sample size (14745) and defined baseline (0.10) and treatment (0.11) conversion rates. Then, perform a Z-test for proportions to compare the conversion rates, interpret the p-value and confidence intervals against an alpha level of 0.05, and summarize the simulation's results and the framework's ability to detect the simulated effect.

## Simulate A/B Test Data

### Subtask:
Generate synthetic A/B test data for control and treatment groups. This will include the number of observations and conversions for each group, based on specified baseline and treatment conversion rates, and the previously calculated sample size.


**Reasoning**:
The subtask requires generating synthetic A/B test data. I will start by setting the necessary variables as instructed, calculating conversions for both control and treatment groups, and then creating a Pandas DataFrame to store this simulated data, including the conversion rates.



In [3]:
n_sample = sample_size_per_group_conv
baseline_rate = baseline_conversion_rate
treatment_rate = baseline_rate + min_detectable_effect

conversions_control = round(n_sample * baseline_rate)
conversions_treatment = round(n_sample * treatment_rate)

ab_test_data = pd.DataFrame({
    'Group': ['Control', 'Treatment'],
    'Observations': [n_sample, n_sample],
    'Conversions': [conversions_control, conversions_treatment]
})

ab_test_data['Conversion_Rate'] = ab_test_data['Conversions'] / ab_test_data['Observations']

print("Simulated A/B Test Data:")
print(ab_test_data)

Simulated A/B Test Data:
       Group  Observations  Conversions  Conversion_Rate
0    Control         14745         1474         0.099966
1  Treatment         14745         1622         0.110003


## Perform Statistical Analysis

### Subtask:
Apply a Z-test for proportions (e.g., using `statsmodels.stats.proportion.proportions_ztest`) to compare the conversion rates between the simulated control and treatment groups. This will yield a p-value indicating the statistical significance of the observed difference.


**Reasoning**:
I need to extract the conversions and observations for both control and treatment groups from the `ab_test_data` DataFrame, then apply `statsmodels.stats.proportion.proportions_ztest` to perform the Z-test and print the results.



In [4]:
from statsmodels.stats.proportion import proportions_ztest

# Extract data for control and treatment groups
conversions_control = ab_test_data[ab_test_data['Group'] == 'Control']['Conversions'].iloc[0]
n_control = ab_test_data[ab_test_data['Group'] == 'Control']['Observations'].iloc[0]

conversions_treatment = ab_test_data[ab_test_data['Group'] == 'Treatment']['Conversions'].iloc[0]
n_treatment = ab_test_data[ab_test_data['Group'] == 'Treatment']['Observations'].iloc[0]

# Perform Z-test for proportions
counts = np.array([conversions_control, conversions_treatment])
nobs = np.array([n_control, n_treatment])

z_statistic, p_value = proportions_ztest(count=counts, nobs=nobs, alternative='two-sided')

print(f"Z-statistic: {z_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

Z-statistic: -2.8116
P-value: 0.0049


**Reasoning**:
Now that the Z-statistic and p-value have been calculated, I will interpret these values in the context of the given alpha level (0.05) to determine the statistical significance of the observed difference. I will also calculate the confidence interval for the difference in proportions, which provides a range of plausible values for the true difference between the control and treatment conversion rates.



In [5]:
alpha = 0.05

print(f"Alpha (significance level): {alpha}")

# Interpretation of p-value
if p_value < alpha:
    print(f"P-value ({p_value:.4f}) is less than alpha ({alpha}), so we reject the null hypothesis.")
    print("This suggests a statistically significant difference in conversion rates between the control and treatment groups.")
else:
    print(f"P-value ({p_value:.4f}) is greater than or equal to alpha ({alpha}), so we fail to reject the null hypothesis.")
    print("This suggests no statistically significant difference in conversion rates between the control and treatment groups.")

# Calculate confidence intervals for the difference in proportions
# Pooled proportion
p_pooled = (conversions_control + conversions_treatment) / (n_control + n_treatment)

# Standard error of the difference for proportions (unpooled, as used by proportions_ztest for confidence interval usually)
# For confidence interval, typically not pooled, but statsmodels proportions_ztest gives a CI directly, let's use the individual proportions.

# Conversion rates
rate_control = conversions_control / n_control
rate_treatment = conversions_treatment / n_treatment

# Difference in rates
diff = rate_treatment - rate_control

# Standard error for the difference (using normal approximation for binomial proportions)
# This calculation needs to be careful, as statsmodels ztest might calculate it differently.
# Let's use the formula for two independent proportions' confidence interval for the difference:
# SE_diff = sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
se_diff = np.sqrt(rate_control * (1 - rate_control) / n_control + rate_treatment * (1 - rate_treatment) / n_treatment)

# Z-score for desired confidence level (e.g., 95% confidence level for alpha=0.05 is 1.96)
z_critical = st.norm.ppf(1 - alpha / 2)

# Margin of error
margin_of_error = z_critical * se_diff

# Confidence interval
conf_interval_lower = diff - margin_of_error
conf_interval_upper = diff + margin_of_error

print(f"\nControl Conversion Rate: {rate_control:.4f}")
print(f"Treatment Conversion Rate: {rate_treatment:.4f}")
print(f"Observed Difference in Conversion Rates (Treatment - Control): {diff:.4f}")
print(f"Z-critical for {((1-alpha)*100):.0f}% confidence: {z_critical:.4f}")
print(f"Margin of Error: {margin_of_error:.4f}")
print(f"Confidence Interval for the difference: [{conf_interval_lower:.4f}, {conf_interval_upper:.4f}]")

# Check if the confidence interval contains zero
if conf_interval_lower < 0 < conf_interval_upper:
    print("The confidence interval includes zero, which would suggest no significant difference.")
else:
    print("The confidence interval does not include zero, which supports a significant difference.")

Alpha (significance level): 0.05
P-value (0.0049) is less than alpha (0.05), so we reject the null hypothesis.
This suggests a statistically significant difference in conversion rates between the control and treatment groups.

Control Conversion Rate: 0.1000
Treatment Conversion Rate: 0.1100
Observed Difference in Conversion Rates (Treatment - Control): 0.0100
Z-critical for 95% confidence: 1.9600
Margin of Error: 0.0070
Confidence Interval for the difference: [0.0030, 0.0170]
The confidence interval does not include zero, which supports a significant difference.


## Summary of Simulation Results and Framework's Ability to Detect Effect

### Statistical Significance:
The A/B test simulation yielded a p-value of **`0.0049`**, which is less than the predetermined significance level (alpha) of **`0.05`**. This leads to the rejection of the null hypothesis. Therefore, the observed difference in conversion rates between the control and treatment groups is statistically significant, meaning it is unlikely to have occurred by random chance.

### Observed Effect:
- **Control Group Conversion Rate**: `0.1000` (10.00%)
- **Treatment Group Conversion Rate**: `0.1100` (11.00%)
- **Observed Difference (Treatment - Control)**: `0.0100` (1.00% absolute increase)

### Confidence Interval:
The 95% confidence interval for the difference in conversion rates is **`[0.0030, 0.0170]`**. This interval does not include zero, further supporting the conclusion that there is a statistically significant difference between the two groups. It suggests that the true difference in conversion rates between the treatment and control groups is likely to be between 0.30% and 1.70% (absolute increase).

### Framework's Ability to Detect the Simulated Effect:
The A/B testing framework, including the sample size calculation and statistical analysis, successfully detected the simulated minimum detectable effect (MDE) of a 1% absolute increase in conversion rate. The `calculate_sample_size` function determined that `14745` observations per group were needed to detect this effect with 80% power and a 5% significance level. The subsequent simulation and Z-test confirmed this, demonstrating that the framework is capable of identifying a true positive effect of the magnitude specified.

In conclusion, the simulation confirms that if a 1% absolute increase in conversion rate truly exists, the A/B test setup (with the calculated sample size) is robust enough to detect it with statistical significance.

## Summary of Simulation Results and Framework's Ability to Detect Effect

### Statistical Significance:
The A/B test simulation yielded a p-value of **`0.0049`**, which is less than the predetermined significance level (alpha) of **`0.05`**. This leads to the rejection of the null hypothesis. Therefore, the observed difference in conversion rates between the control and treatment groups is statistically significant, meaning it is unlikely to have occurred by random chance.

### Observed Effect:
- **Control Group Conversion Rate**: `0.1000` (10.00%)
- **Treatment Group Conversion Rate**: `0.1100` (11.00%)
- **Observed Difference (Treatment - Control)**: `0.0100` (1.00% absolute increase)

### Confidence Interval:
The 95% confidence interval for the difference in conversion rates is **`[0.0030, 0.0170]`**. This interval does not include zero, further supporting the conclusion that there is a statistically significant difference between the two groups. It suggests that the true difference in conversion rates between the treatment and control groups is likely to be between 0.30% and 1.70% (absolute increase).

### Framework's Ability to Detect the Simulated Effect:
The A/B testing framework, including the sample size calculation and statistical analysis, successfully detected the simulated minimum detectable effect (MDE) of a 1% absolute increase in conversion rate. The `calculate_sample_size` function determined that `14745` observations per group were needed to detect this effect with 80% power and a 5% significance level. The subsequent simulation and Z-test confirmed this, demonstrating that the framework is capable of identifying a true positive effect of the magnitude specified.

In conclusion, the simulation confirms that if a 1% absolute increase in conversion rate truly exists, the A/B test setup (with the calculated sample size) is robust enough to detect it with statistical significance.

## Summary:

### Data Analysis Key Findings
*   **Simulated Data:** The control group had 14,745 observations with 1,474 conversions, resulting in a conversion rate of approximately 0.1000 (10.00%). The treatment group also had 14,745 observations with 1,622 conversions, yielding a conversion rate of approximately 0.1100 (11.00%).
*   **Observed Effect:** The absolute difference in conversion rates between the treatment and control groups was 0.0100 (1.00% increase).
*   **Z-test Results:** A Z-statistic of -2.8116 and a p-value of 0.0049 were obtained from the proportions Z-test.
*   **Statistical Significance:** Since the p-value (0.0049) is less than the alpha level of 0.05, the null hypothesis was rejected, indicating a statistically significant difference in conversion rates between the groups.
*   **Confidence Interval:** The 95% confidence interval for the difference in conversion rates is [0.0030, 0.0170]. This interval does not include zero, reinforcing the conclusion of a significant difference, suggesting the true difference lies between 0.30% and 1.70% absolute increase.
*   **Framework's Efficacy:** The A/B testing framework, utilizing the calculated sample size of 14,745 per group, successfully detected the simulated 1% absolute increase in conversion rate, validating its ability to identify a true positive effect of the specified magnitude.

### Insights or Next Steps
*   The A/B test setup, including the sample size calculation, effectively identified the simulated minimum detectable effect, confirming its robustness for future experiments with similar parameters.
*   Proceed with implementing the A/B test in a real-world scenario, confident that the experiment design is capable of detecting a 1% absolute uplift in conversion rate if it exists.
